In [1]:
import numpy as np
import os
import pretty_midi
import mir_eval

import ByteDanceEncapsulation as bd

In [2]:
def compute_score_this_piano(piano_name, time_limit = 30):
    scores = []
    accs = []
    for file in os.listdir(f"{bd.path_maps}/{piano_name}/MUS"):
        if "wav" in file:
            ref_intervals, ref_pitches = bd.load_ref_in_arrays(f"{bd.path_maps}/{piano_name}/MUS/{file.replace('wav', 'txt')}", time_limit = time_limit)
            est_intervals = []
            est_pitches = []
            try: # If already computed
                os.open(f"{bd.path_outputs}/{piano_name}/{file.replace('.wav', '')}.mid", os.O_RDONLY)
            except FileNotFoundError:
                bd.run_estimates_file(f"{bd.path_maps}/{piano_name}/MUS/{file}", piano_name)
                
            pm = pretty_midi.PrettyMIDI(f"{bd.path_outputs}/{piano_name}/{file.replace('.wav', '')}.mid")
            for instrument in pm.instruments:
                for note in instrument.notes:
                    start = note.start
                    if (time_limit == None) or ((time_limit != None) and (start < time_limit)):
                        end = note.end
                        est_intervals.append([start, end])
                        pitch = note.pitch
                        est_pitches.append(pitch)
            est_intervals = np.array(est_intervals)
            est_pitches = np.array(est_pitches)
            (prec, rec, f_mes, _) = mir_eval.transcription.precision_recall_f1_overlap(ref_intervals, ref_pitches, est_intervals, est_pitches, onset_tolerance=0.05, pitch_tolerance=50.0, offset_ratio=None, strict=False, beta=1.0)
            scores.append([prec, rec, f_mes])
            matching = mir_eval.transcription.match_notes(ref_intervals, ref_pitches, est_intervals,est_pitches, onset_tolerance=0.05,pitch_tolerance=50.0,offset_ratio=None)
            TP = len(matching)
            try:
                FP = int(TP * (1 - prec) / prec)
            except ZeroDivisionError:
                FP = 0
            try:
                FN = int(TP * (1 - rec) / rec)
            except ZeroDivisionError:
                FN = 0
            try:
                accuracy = TP/(TP + FP + FN)
            except ZeroDivisionError:
                accuracy = 0
            accs.append(accuracy)
    return [np.mean(np.array(scores)[:,i]) for i in range(3)], np.mean(accs)

In [3]:
list_pianos = ["AkPnBcht", "AkPnBsdf", "AkPnCGdD", "AkPnStgb", "ENSTDkAm", "ENSTDkCl", "SptkBGAm", "SptkBGCl", "StbgTGd2"]
for piano in list_pianos:
    print(piano)
    scores, acc = compute_score_this_piano(piano, time_limit = 30)
    print(f"30sec - Prec: {round(scores[0],4)}, Recall: {round(scores[1],4)}, F measure: {round(scores[2],4)}, accuracy: {round(acc,4)}")
    
    #scores, acc = compute_score_this_piano(piano, time_limit = None)
    #print(f"Without limit - Prec: {scores[0]}, Recall: {scores[1]}, F measure: {scores[2]}, accuracy: {acc}")

AkPnBcht
30sec - Prec: 0.985, Recall: 0.9782, F measure: 0.9813, accuracy: 0.9669
AkPnBsdf
30sec - Prec: 0.9341, Recall: 0.9603, F measure: 0.9465, accuracy: 0.9032
AkPnCGdD
30sec - Prec: 0.9809, Recall: 0.9742, F measure: 0.9772, accuracy: 0.9607
AkPnStgb
30sec - Prec: 0.8174, Recall: 0.9256, F measure: 0.8657, accuracy: 0.7703
ENSTDkAm
30sec - Prec: 0.7156, Recall: 0.8416, F measure: 0.772, accuracy: 0.6522
ENSTDkCl
30sec - Prec: 0.8678, Recall: 0.9231, F measure: 0.8932, accuracy: 0.8177
SptkBGAm
30sec - Prec: 0.982, Recall: 0.9768, F measure: 0.9792, accuracy: 0.9633
SptkBGCl
30sec - Prec: 0.9688, Recall: 0.9778, F measure: 0.973, accuracy: 0.9511
StbgTGd2
30sec - Prec: 0.9316, Recall: 0.9623, F measure: 0.9458, accuracy: 0.9044
